In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

train.head

<bound method NDFrame.head of          id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this #earthquake M...       1  
1                Forest fire near La Ronge Sask. Canada       1  
2     All residents asked to 'shelter in place' are ...       1  
3     13,000 people receive #wildfires evacuation or...       1  
4     Just got sent this photo from Ruby #Alaska as ...       1  
...                                                 ...     ...  
7608  Two giant cranes holding a bridge collapse int...       1  
7609  @aria_ahrary @TheTawniest The out of control w...  

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

english_stopwords = stopwords.words('english')
vocab_size = 5000
stop_words = set(stopwords.words('english'))

def filter_stop_words(text):# convert text to string
    text = str(text)
 # tokenize the text into words
    words = word_tokenize(text)
    # filter out the stop words
    filtered_words = [w for w in words if w.lower() not in stop_words]
    # join the filtered words back into a string
    filtered_text = ' '.join(filtered_words)
    # print(type(filtered_text))
    return filtered_text

# apply the custom function to the text column
train['text'] = train['text'].apply(filter_stop_words)
test['text'] = test['text'].apply(filter_stop_words)

X_train = train['text'].to_numpy()
y_train = train['target'].to_numpy()
X_test = test['text'].to_numpy()
# print(X_train)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=vocab_size)

vectorizer.fit(train['text'])
vocab = vectorizer.vocabulary_

# print(vocab)

# fit the vectorizer on the series and get the feature names
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

X_train = X_train.toarray()
X_test = X_test.toarray()

print(X_train.shape)

(7613, 5000)


In [4]:
import torch

print(X_train.shape)
print(y_train.shape)

X_train = torch.tensor(X_train, dtype=torch.int32)
y_train = torch.tensor(y_train, dtype=torch.int32)
X_test = torch.tensor(X_test, dtype=torch.int32)

print(X_train)
print(y_train)

(7613, 5000)
(7613,)
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)
tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.int32)


In [5]:
import torch.nn as nn
import torch.optim as optim

class LSTMClassifier(nn.Module):
    def __init__(self):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, 10)
        self.lstm = nn.LSTM(10, 5)
        self.linear = nn.Linear(5, 2)
        self.softmax = nn.Softmax(dim=1) # Softmax activation function
    
    def forward(self, x):
        x = self.embedding(x) # Apply embedding layer
        x, _ = self.lstm(x) # Apply LSTM layer and get the last hidden state
        x = x[-1] # Get the last output of the LSTM
        x = self.linear(x) # Apply linear layer
        x = self.softmax(x) # Apply softmax activation
        return x

# Create an instance of the model
model = LSTMClassifier()

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss() # Cross entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.01) # Adam optimizer

# Train the model for 100 epochs
epochs = 100
for epoch in range(epochs):
    # Forward pass
    y_pred = model(X_train) # Predict outputs
    loss = criterion(y_pred, y_train) # Compute loss
    
    # Backward pass and update
    optimizer.zero_grad() # Zero the gradients
    loss.backward() # Compute gradients
    optimizer.step() # Update parameters
    
    # Print loss every 10 epochs
    # if (epoch + 1) % 10 == 0:
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

# # Test the model
# with torch.no_grad(): # No need to compute gradients for testing
#     y_pred = model(X_val) # Predict outputs
#     y_pred = torch.argmax(y_pred, dim=1) # Get the predicted labels
#     accuracy = (y_pred == y_val).float().mean() # Compute accuracy
#     print(f"Accuracy: {accuracy:.4f}")

ValueError: Expected input batch_size (5000) to match target batch_size (7613).